# A Loan Data with Numpy

# Importing the packages

In [834]:
import pandas as pd
import numpy as np

In [835]:
#improve the printing options,improve array’s readability
np.set_printoptions(suppress = True, linewidth =100, precision = 2) #suppress = True, prevents python from using scientific options for numbers

In [836]:
#importing the dataset
raw_data = np.genfromtxt('loan-data.csv', 
                         delimiter = ';',
                         skip_header = 1,
                         autostrip =True)
raw_data

array([[48010226.  ,         nan,    35000.  , ...,         nan,         nan,     9452.96],
       [57693261.  ,         nan,    30000.  , ...,         nan,         nan,     4679.7 ],
       [59432726.  ,         nan,    15000.  , ...,         nan,         nan,     1969.83],
       ...,
       [50415990.  ,         nan,    10000.  , ...,         nan,         nan,     2185.64],
       [46154151.  ,         nan,         nan, ...,         nan,         nan,     3199.4 ],
       [66055249.  ,         nan,    10000.  , ...,         nan,         nan,      301.9 ]])

In [837]:
np.isnan(raw_data).sum()

88005

The nanmean() and nanmax() functions in NumPy are used to compute the mean and the maximum of an array while ignoring any NaN 

In [838]:
#create a filler for all the missing entries in the dataset
temporary_fill = np.nanmax(raw_data) + 1
#create a variable wich will hold the mean for every columns
temporary_mean = np.nanmean(raw_data, axis = 0)

C:\Users\User\AppData\Local\Temp\ipykernel_13396\2730524203.py:4: RuntimeWarning: Mean of empty slice
  temporary_mean = np.nanmean(raw_data, axis = 0)


In [839]:
temporary_stats = np.array([np.nanmin(raw_data, axis = 0), 
                           temporary_mean,
                           np.nanmax(raw_data, axis = 0)])

C:\Users\User\AppData\Local\Temp\ipykernel_13396\2832593045.py:1: RuntimeWarning: All-NaN slice encountered
  temporary_stats = np.array([np.nanmin(raw_data, axis = 0),
C:\Users\User\AppData\Local\Temp\ipykernel_13396\2832593045.py:3: RuntimeWarning: All-NaN slice encountered
  np.nanmax(raw_data, axis = 0)])


In [840]:
temporary_stats

array([[  373332.  ,         nan,     1000.  ,         nan,     1000.  ,         nan,        6.  ,
              31.42,         nan,         nan,         nan,         nan,         nan,        0.  ],
       [54015809.19,         nan,    15273.46,         nan,    15311.04,         nan,       16.62,
             440.92,         nan,         nan,         nan,         nan,         nan,     3143.85],
       [68616519.  ,         nan,    35000.  ,         nan,    35000.  ,         nan,       28.99,
            1372.97,         nan,         nan,         nan,         nan,         nan,    41913.62]])

# Split the columns into strings and numeric 

In [841]:
#Split the columns into strings and numeric and return the indices
columns_strings = np.argwhere(np.isnan(temporary_mean)).squeeze()
columns_numeric = np.argwhere(np.isnan(temporary_mean)==False).squeeze()
columns_numeric,columns_strings


(array([ 0,  2,  4,  6,  7, 13], dtype=int64),
 array([ 1,  3,  5,  8,  9, 10, 11, 12], dtype=int64))

# Re-importing the data 


In [842]:
loan_data_strings = np.genfromtxt('loan-data.csv', 
                         delimiter = ';',
                         skip_header = 1,
                         autostrip =True,
                         usecols = columns_strings,
                         dtype = str )


loan_data_numeric = np.genfromtxt('loan-data.csv', 
                         delimiter = ';',
                         skip_header = 1,
                         autostrip =True,
                         usecols = columns_numeric,
                         filling_values = temporary_fill )

#check any missing values
np.isnan(loan_data_numeric).sum()



0

In [774]:
#import the header
header_full = np.genfromtxt('loan-data.csv', 
                         delimiter = ';',
                         skip_footer = raw_data.shape[0],
                         autostrip =True,
                         dtype = str )

header_full

array(['id', 'issue_d', 'loan_amnt', 'loan_status', 'funded_amnt', 'term', 'int_rate',
       'installment', 'grade', 'sub_grade', 'verification_status', 'url', 'addr_state',
       'total_pymnt'], dtype='<U19')

In [843]:
#separate header_strings and header-numeric
header_strings, header_numeric = header_full[columns_strings], header_full[columns_numeric]

header_strings, header_numeric

(array(['issue_d', 'loan_status', 'term', 'grade', 'sub_grade', 'verification_status', 'url',
        'addr_state'], dtype='<U19'),
 array(['id', 'loan_amnt', 'funded_amnt', 'int_rate', 'installment', 'total_pymnt'], dtype='<U19'))

# Creating Checkpoints

In [844]:
#we create checkpoints where we could store our dataset progress without to avoid the risque of loosing it
def checkpoint(file_name, checkpoint_head, checkpoint_data):
    np.savez(file_name, header = checkpoint_head , data = checkpoint_data)
    checkpoint_variable = np.load(file_name + ".npz")
    return (checkpoint_variable)

In [845]:
checkpoint_test = checkpoint("checkpoint_test1", header_strings, loan_data_strings )

#lets check our checkpoints
checkpoint_test['header'] == header_strings
np.array_equal(checkpoint_test['data'], loan_data_strings )
header_strings

array(['issue_d', 'loan_status', 'term', 'grade', 'sub_grade', 'verification_status', 'url',
       'addr_state'], dtype='<U19')

In [778]:
# Revert to the checkpoint
checkpoint_data = np.load('checkpoint_test1.npz')
header = checkpoint_data['header']
data = checkpoint_data['data']

# Manipulating String Data

In [846]:
#Lets change the first column name to a more meanfull name 
header_strings[0] = "issue_date"
header_strings

array(['issue_date', 'loan_status', 'term', 'grade', 'sub_grade', 'verification_status', 'url',
       'addr_state'], dtype='<U19')

# Transform Issue Date to numbers

In [847]:
#It's always better to have a date column as integer rather than strings 
np.unique(loan_data_strings[:,0])


array(['', 'Apr-15', 'Aug-15', 'Dec-15', 'Feb-15', 'Jan-15', 'Jul-15', 'Jun-15', 'Mar-15',
       'May-15', 'Nov-15', 'Oct-15', 'Sep-15'], dtype='<U69')

In [848]:
# I will strip the excess data -15 because its not relevant as all columns are year 15
loan_data_strings[:,0] = np.char.strip(loan_data_strings[:,0],'-15')
loan_data_strings[:,0]

array(['May', '', 'Sep', ..., 'Jun', 'Apr', 'Dec'], dtype='<U69')

In [849]:
#Represent month values as integers as strings take more memory space
months = np.array(['', 'Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'])
months

array(['', 'Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'],
      dtype='<U3')

In [850]:
#Lets convert the entire issue date column to numbers

for i in range(13): 
    loan_data_strings[:,0] = np.where(loan_data_strings[:,0] == months[i], 
                                      i ,
                                      loan_data_strings[:,0])
np.unique(loan_data_strings[:,0])   

array(['0', '1', '10', '11', '12', '2', '3', '4', '5', '6', '7', '8', '9'], dtype='<U69')

In [851]:
checkpoint_test = checkpoint("checkpoint_test2", header_strings, loan_data_strings )

#lets check our checkpoints
checkpoint_test['header'] == header_strings
np.array_equal(checkpoint_test['data'], loan_data_strings )
header_strings

array(['issue_date', 'loan_status', 'term', 'grade', 'sub_grade', 'verification_status', 'url',
       'addr_state'], dtype='<U19')

# Transform Loan status into dummy variable

In [852]:
#start with the loan status header than the data
header_strings[1]


'loan_status'

In [853]:
#Examin the data in loan status
np.unique(loan_data_strings[:,1])

array(['', 'Charged Off', 'Current', 'Default', 'Fully Paid', 'In Grace Period', 'Issued',
       'Late (16-30 days)', 'Late (31-120 days)'], dtype='<U69')

In [787]:
np.unique(loan_data_strings[:,1]).size

9

array(['1', '1', '1', ..., '1', '1', '1'], dtype='<U69')

In [854]:
#we notice that we can categorize the loan status column into two categories '0' for 'bad' accounts and '1' to indicate 'good' account

bad_status = np.array(['', 'Charged Off', 'Default', 'Late (31-120 days)'])
good_status = np.array(['Current','Issued', 'Fully Paid', 'In Grace Period', 'Late (16-30 days)'])

'''for i in range(loan_data_strings[:,1].size):
    for j in range(4):
      if loan_data_strings[:,1][i] == bad_status[j]:
         loan_data_strings[:,1][i] = 0
      else:
         continue
            
for i in range(loan_data_strings[:,1].size):
    for j in range(5):
      if loan_data_strings[:,1][i] == good_status[j]:
         loan_data_strings[:,1][i] = 1
      else:
         continue
'''
#we could have used the where() method to do all the above work in one single line
#numpy.where(condition, x, y) x if condition satisfied, y if not
loan_data_strings[:,1] = np.where(np.isin(loan_data_strings[:,1], bad_status), 0, 1)


In [855]:
np.unique(loan_data_strings[:,1])


array(['0', '1'], dtype='<U69')

In [856]:
checkpoint_test = checkpoint("checkpoint_test3", header_strings, loan_data_strings )

#lets check our checkpoints
checkpoint_test['header'] == header_strings
np.array_equal(checkpoint_test['data'], loan_data_strings )
header_strings

array(['issue_date', 'loan_status', 'term', 'grade', 'sub_grade', 'verification_status', 'url',
       'addr_state'], dtype='<U19')

# Transform Term into dummy variable

In [793]:
#Lets examin Term column data and header
header_strings


array(['issue_date', 'loan_status', 'term', 'grade', 'sub_grade', 'verification_status', 'url',
       'addr_state'], dtype='<U19')

In [857]:
loan_data_strings[:,2]

array(['36 months', '36 months', '36 months', ..., '36 months', '36 months', '36 months'],
      dtype='<U69')

In [858]:
np.unique(loan_data_strings[:,2])

array(['', '36 months', '60 months'], dtype='<U69')

In [859]:
#I suggest striping 'months' string 
loan_data_strings[:,2] = np.char.strip(loan_data_strings[:,2],' months')
loan_data_strings[:,2]

array(['36', '36', '36', ..., '36', '36', '36'], dtype='<U69')

In [860]:
np.unique(loan_data_strings[:,2])

array(['', '36', '60'], dtype='<U69')

In [861]:
#lets now change column name to a more descriptive one
header_strings[2] = 'term_months'
header_strings[2] 

'term_months'

In [862]:
#now we need to fill in the missiing values in '' we will replace it with '60'
#we choose '60' because in CRM if a value is missing we assume the WORST senario

#numpy.where(condition, x, y) x if condition satisfied, y if not
loan_data_strings[:,2] = np.where(loan_data_strings[:,2] == '', 
                                  '60',
                                  loan_data_strings[:,2])
np.unique(loan_data_strings[:,2])


array(['36', '60'], dtype='<U69')

In [863]:
checkpoint_test = checkpoint("checkpoint_test4", header_strings, loan_data_strings )

#lets check our checkpoints
checkpoint_test['header'] == header_strings
np.array_equal(checkpoint_test['data'], loan_data_strings )

True

In [864]:
checkpoint_test = checkpoint("checkpoint_test5", header_strings, loan_data_strings )

#lets check our checkpoints
checkpoint_test['header'] == header_strings
np.array_equal(checkpoint_test['data'], loan_data_strings )
header_strings

array(['issue_date', 'loan_status', 'term_months', 'grade', 'sub_grade', 'verification_status',
       'url', 'addr_state'], dtype='<U19')

# Examin Grade and Sub Grade variable

In [865]:
header_strings[3] 

'grade'

In [866]:
loan_data_strings[:,3]

array(['C', 'A', 'B', ..., 'A', 'D', 'A'], dtype='<U69')

In [869]:
np.unique(loan_data_strings[:,3])

array(['', 'A', 'B', 'C', 'D', 'E', 'F', 'G'], dtype='<U69')

In [868]:
#lets examin sub grade before we decide what to do
np.unique(loan_data_strings[:,4])

array(['', 'A1', 'A2', 'A3', 'A4', 'A5', 'B1', 'B2', 'B3', 'B4', 'B5', 'C1', 'C2', 'C3', 'C4',
       'C5', 'D1', 'D2', 'D3', 'D4', 'D5', 'E1', 'E2', 'E3', 'E4', 'E5', 'F1', 'F2', 'F3', 'F4',
       'F5', 'G1', 'G2', 'G3', 'G4', 'G5'], dtype='<U69')

In [806]:
#looks like sub grade contains the information in grade plus more detail
#here we face redandacy we should remove the column grade from our datset
#before we delete grade we will use 'grade' to fill in some of the missing values in 'sub grade'

#lets see how many '' we have
np.unique(loan_data_strings[:,4], return_counts = True)
# we have 514 columns with empty values that a lot to withdraw from the data.
#lets try to fill in the subgrade using the grade column 


(array(['', 'A1', 'A2', 'A3', 'A4', 'A5', 'B1', 'B2', 'B3', 'B4', 'B5', 'C1', 'C2', 'C3', 'C4',
        'C5', 'D1', 'D2', 'D3', 'D4', 'D5', 'E1', 'E2', 'E3', 'E4', 'E5', 'F1', 'F2', 'F3', 'F4',
        'F5', 'G1', 'G2', 'G3', 'G4', 'G5'], dtype='<U69'),
 array([514, 285, 278, 239, 323, 502, 509, 517, 530, 553, 494, 629, 567, 586, 564, 423, 391, 267,
        250, 255, 223, 235, 162, 171, 139, 114,  94,  52,  34,  43,  16,  19,  10,   3,   7,   2],
       dtype=int64))

In [871]:
for i in np.unique(loan_data_strings[:,3])[1:]:
  loan_data_strings[:,4] = np.where((loan_data_strings[:,4] == '') & (loan_data_strings[:,3] == i), 
                                  i + '5', #we assume the worst
                                  loan_data_strings[:,4])


np.unique(loan_data_strings[:,4])

array(['', 'A1', 'A2', 'A3', 'A4', 'A5', 'B1', 'B2', 'B3', 'B4', 'B5', 'C1', 'C2', 'C3', 'C4',
       'C5', 'D1', 'D2', 'D3', 'D4', 'D5', 'E1', 'E2', 'E3', 'E4', 'E5', 'F1', 'F2', 'F3', 'F4',
       'F5', 'G1', 'G2', 'G3', 'G4', 'G5'], dtype='<U69')

In [872]:
np.unique(loan_data_strings[:,4], return_counts = True)

(array(['', 'A1', 'A2', 'A3', 'A4', 'A5', 'B1', 'B2', 'B3', 'B4', 'B5', 'C1', 'C2', 'C3', 'C4',
        'C5', 'D1', 'D2', 'D3', 'D4', 'D5', 'E1', 'E2', 'E3', 'E4', 'E5', 'F1', 'F2', 'F3', 'F4',
        'F5', 'G1', 'G2', 'G3', 'G4', 'G5'], dtype='<U69'),
 array([  9, 285, 278, 239, 323, 592, 509, 517, 530, 553, 633, 629, 567, 586, 564, 577, 391, 267,
        250, 255, 288, 235, 162, 171, 139, 160,  94,  52,  34,  43,  24,  19,  10,   3,   7,   5],
       dtype=int64))

In [873]:
#we still have 9 empty columns, because there is empty subgrades without grades
#as they all only 9 columns out of a total of 1000 columns we can delete them
#but we will not, we opt of adding a new grade 'H1' worse than G5 and put all missing values
for i in np.unique(loan_data_strings[:,3])[1:]:
   loan_data_strings[:,4] = np.where(loan_data_strings[:,4] == '', 
                                  'H1', 
                                  loan_data_strings[:,4])


In [874]:
np.unique(loan_data_strings[:,4])

array(['A1', 'A2', 'A3', 'A4', 'A5', 'B1', 'B2', 'B3', 'B4', 'B5', 'C1', 'C2', 'C3', 'C4', 'C5',
       'D1', 'D2', 'D3', 'D4', 'D5', 'E1', 'E2', 'E3', 'E4', 'E5', 'F1', 'F2', 'F3', 'F4', 'F5',
       'G1', 'G2', 'G3', 'G4', 'G5', 'H1'], dtype='<U69')

In [875]:
checkpoint_test = checkpoint("checkpoint_test6", header_strings, loan_data_strings )

#lets check our checkpoints
checkpoint_test['header'] == header_strings
np.array_equal(checkpoint_test['data'], loan_data_strings )
header_strings

array(['issue_date', 'loan_status', 'term_months', 'grade', 'sub_grade', 'verification_status',
       'url', 'addr_state'], dtype='<U19')

# Removing Grade column

In [876]:
#we dont need grade anymore as sub grade now contain all the information in grade 
#loan_data_strings = np.delete(loan_data_strings, 3, axis = 1)

I had to comment delete grade so I don't execute it again by mistake


In [877]:
loan_data_strings[:,3]

array(['C3', 'A5', 'B5', ..., 'A5', 'D2', 'A4'], dtype='<U69')

In [878]:
#header_strings = np.delete(header_strings, 3) # will remove the 4th element


In [879]:
header_strings

array(['issue_date', 'loan_status', 'term_months', 'sub_grade', 'verification_status', 'url',
       'addr_state'], dtype='<U19')

In [880]:
checkpoint_test = checkpoint("checkpoint_test7", header_strings, loan_data_strings )

#lets check our checkpoints
checkpoint_test['header'] == header_strings
np.array_equal(checkpoint_test['data'], loan_data_strings )
header_strings

array(['issue_date', 'loan_status', 'term_months', 'sub_grade', 'verification_status', 'url',
       'addr_state'], dtype='<U19')

# Converting sub_grade

In [710]:
#Lests go back to our checkpoint checkpoint_test7
#loan_data_strings = data
#header_strings = header


In [882]:
np.unique(loan_data_strings[:,3])

array(['A1', 'A2', 'A3', 'A4', 'A5', 'B1', 'B2', 'B3', 'B4', 'B5', 'C1', 'C2', 'C3', 'C4', 'C5',
       'D1', 'D2', 'D3', 'D4', 'D5', 'E1', 'E2', 'E3', 'E4', 'E5', 'F1', 'F2', 'F3', 'F4', 'F5',
       'G1', 'G2', 'G3', 'G4', 'G5', 'H1'], dtype='<U69')

In [831]:
np.unique(loan_data_strings[:,3]).size

36

In [883]:
#instead of using where() function 36 times to associate a unique value for each grade
#we will create a dictionary with the keys are the grades and values the associated numbers

key = list(np.unique(loan_data_strings[:,3]))
#len(key) = 36
values = list(range(1, len(key)+1))
sub_grade_dict = dict(zip(key,values))
sub_grade_dict 

{'A1': 1,
 'A2': 2,
 'A3': 3,
 'A4': 4,
 'A5': 5,
 'B1': 6,
 'B2': 7,
 'B3': 8,
 'B4': 9,
 'B5': 10,
 'C1': 11,
 'C2': 12,
 'C3': 13,
 'C4': 14,
 'C5': 15,
 'D1': 16,
 'D2': 17,
 'D3': 18,
 'D4': 19,
 'D5': 20,
 'E1': 21,
 'E2': 22,
 'E3': 23,
 'E4': 24,
 'E5': 25,
 'F1': 26,
 'F2': 27,
 'F3': 28,
 'F4': 29,
 'F5': 30,
 'G1': 31,
 'G2': 32,
 'G3': 33,
 'G4': 34,
 'G5': 35,
 'H1': 36}

In [884]:
# Iterate over unique values in the 4th column
for i in np.unique(loan_data_strings[:, 3]):
    # Create a mask for the current unique value
    mask = loan_data_strings[:, 3] == i
    # Update the values in the 4th column using the mask
    loan_data_strings[mask, 3] = sub_grade_dict[i]
    
loan_data_strings[:, 3]    

array(['13', '5', '10', ..., '5', '17', '4'], dtype='<U69')

In [885]:
np.unique(loan_data_strings[:, 3])

array(['1', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '2', '20', '21', '22',
       '23', '24', '25', '26', '27', '28', '29', '3', '30', '31', '32', '33', '34', '35', '36',
       '4', '5', '6', '7', '8', '9'], dtype='<U69')

In [886]:
checkpoint_test = checkpoint("checkpoint_test8", header_strings, loan_data_strings )

#lets check our checkpoints
checkpoint_test['header'] == header_strings
np.array_equal(checkpoint_test['data'], loan_data_strings )
header_strings

array(['issue_date', 'loan_status', 'term_months', 'sub_grade', 'verification_status', 'url',
       'addr_state'], dtype='<U19')

# Verification status 

In [824]:
# Revert to the checkpoint
#checkpoint_data = np.load('checkpoint_test8.npz')
#header = checkpoint_data['header']
#data = checkpoint_data['data']

In [825]:
#loan_data_strings = data
#header_strings = header


In [887]:
np.unique(loan_data_strings[:,4])

array(['', 'Not Verified', 'Source Verified', 'Verified'], dtype='<U69')

In [888]:
header_strings[4]

'verification_status'

In [889]:
loan_data_strings[:,4]

array(['Verified', 'Source Verified', 'Verified', ..., 'Source Verified', 'Source Verified', ''],
      dtype='<U69')

In [890]:
np.unique(loan_data_strings[:,4])

array(['', 'Not Verified', 'Source Verified', 'Verified'], dtype='<U69')

In [891]:
#as we assume the worst will attribute '0' to empty and 'Not Verified'
#and the value '1' to 'Source Verified', 'Verified'

# Create a mask for the conditions to be '0'
mask_0 = (loan_data_strings[:, 4] == '') | (loan_data_strings[:, 4] == 'Not Verified')

# Apply the mask to set values to '0'
loan_data_strings[mask_0, 4] = 0

# Set the remaining values to '1'
loan_data_strings[~mask_0, 4] = 1

# Convert the column to integers
loan_data_strings[:, 4] = loan_data_strings[:, 4].astype(int) # cast type into int

#print(loan_data_strings)
print(np.unique(loan_data_strings[:, 4]))

['0' '1']


In [892]:
np.unique(loan_data_strings[:,4])

array(['0', '1'], dtype='<U69')

In [893]:
checkpoint_test = checkpoint("checkpoint_test9", header_strings, loan_data_strings )

#lets check our checkpoints
checkpoint_test['header'] == header_strings
np.array_equal(checkpoint_test['data'], loan_data_strings )

True

# URL

In [894]:
np.unique(loan_data_strings[:,5])

array(['https://www.lendingclub.com/browse/loanDetail.action?loan_id=12606806',
       'https://www.lendingclub.com/browse/loanDetail.action?loan_id=13026045',
       'https://www.lendingclub.com/browse/loanDetail.action?loan_id=1312426', ...,
       'https://www.lendingclub.com/browse/loanDetail.action?loan_id=8138291',
       'https://www.lendingclub.com/browse/loanDetail.action?loan_id=8214572',
       'https://www.lendingclub.com/browse/loanDetail.action?loan_id=849994'], dtype='<U69')

In [895]:
#We can notice that the url string values is identical
#we can stripe this identical value
loan_data_strings[:,5] = np.char.strip(loan_data_strings[:,5] , 'https://www.lendingclub.com/browse/loanDetail.action?loan_id=')

In [896]:
np.unique(loan_data_strings[:,5])

array(['12606806', '13026045', '1312426', ..., '8138291', '8214572', '849994'], dtype='<U69')

In [900]:
#We notice that the values remaing are the IDs values
#Lets check with np.array_equal if the numeric values are equal 
np.array_equal(loan_data_strings[:,5].astype(int),loan_data_numeric[:,0].astype(int))

True

In [902]:
strings = loan_data_strings[:,5].astype(int)
strings
    

array([48010226, 57693261, 59432726, ..., 50415990, 46154151, 66055249])

In [903]:
loan_data_strings[:,5]

array(['48010226', '57693261', '59432726', ..., '50415990', '46154151', '66055249'], dtype='<U69')

In [907]:
#As the id column values and the URL column values are identical
#we could remove URL 

#loan_data_strings = np.delete(loan_data_strings, 5, axis = 1)



I had commented remove URL so I don't execute it twice or more 

In [908]:
#header_strings = np.delete(header_strings, 5)

In [909]:
header_strings

array(['issue_date', 'loan_status', 'term_months', 'sub_grade', 'verification_status',
       'addr_state'], dtype='<U19')

In [913]:
header_strings[5] = "state_address"

In [911]:
np.unique(loan_data_strings[:,5])

array(['', 'AK', 'AL', 'AR', 'AZ', 'CA', 'CO', 'CT', 'DC', 'DE', 'FL', 'GA', 'HI', 'IL', 'IN',
       'KS', 'KY', 'LA', 'MA', 'MD', 'ME', 'MI', 'MN', 'MO', 'MS', 'MT', 'NC', 'ND', 'NE', 'NH',
       'NJ', 'NM', 'NV', 'NY', 'OH', 'OK', 'OR', 'PA', 'RI', 'SC', 'SD', 'TN', 'TX', 'UT', 'VA',
       'VT', 'WA', 'WI', 'WV', 'WY'], dtype='<U69')

In [914]:
states_names, states_count = np.unique(loan_data_strings[:,5], return_counts = True)
states_count_sorted = np.argsort(-states_count)
states_names[states_count_sorted], states_count[states_count_sorted]

(array(['CA', 'NY', 'TX', 'FL', '', 'IL', 'NJ', 'GA', 'PA', 'OH', 'MI', 'NC', 'VA', 'MD', 'AZ',
        'WA', 'MA', 'CO', 'MO', 'MN', 'IN', 'WI', 'CT', 'TN', 'NV', 'AL', 'LA', 'OR', 'SC', 'KY',
        'KS', 'OK', 'UT', 'AR', 'MS', 'NH', 'NM', 'WV', 'HI', 'RI', 'MT', 'DE', 'DC', 'WY', 'AK',
        'NE', 'SD', 'VT', 'ND', 'ME'], dtype='<U69'),
 array([1336,  777,  758,  690,  500,  389,  341,  321,  320,  312,  267,  261,  242,  222,  220,
         216,  210,  201,  160,  156,  152,  148,  143,  143,  130,  119,  116,  108,  107,   84,
          84,   83,   74,   74,   61,   58,   57,   49,   44,   40,   28,   27,   27,   27,   26,
          25,   24,   17,   16,   10], dtype=int64))

In [912]:
# we can notice that the address state is the first two inittials of the 50 USA states


In [915]:
loan_data_strings[:,5] = np.where(loan_data_strings[:,5] == '', 
                                  0, 
                                  loan_data_strings[:,5])

In [916]:
states_west = np.array(['WA', 'OR','CA','NV','ID','MT', 'WY','UT','CO', 'AZ','NM','HI','AK'])
states_south = np.array(['TX','OK','AR','LA','MS','AL','TN','KY','FL','GA','SC','NC','VA','WV','MD','DE','DC'])
states_midwest = np.array(['ND','SD','NE','KS','MN','IA','MO','WI','IL','IN','MI','OH'])
states_east = np.array(['PA','NY','NJ','CT','MA','VT','NH','ME','RI'])

https://www2.census.gov/geo/pdfs/maps-data/maps/reference/us_regdiv.pdf

In [917]:
loan_data_strings[:,5] = np.where(np.isin(loan_data_strings[:,5], states_west), 1, loan_data_strings[:,5])
loan_data_strings[:,5] = np.where(np.isin(loan_data_strings[:,5], states_south), 2, loan_data_strings[:,5])
loan_data_strings[:,5] = np.where(np.isin(loan_data_strings[:,5], states_midwest), 3, loan_data_strings[:,5])
loan_data_strings[:,5] = np.where(np.isin(loan_data_strings[:,5], states_east), 4, loan_data_strings[:,5])

In [918]:
np.unique(loan_data_strings[:,5])

array(['0', '1', '2', '3', '4'], dtype='<U69')

# Converting to Numbers

In [919]:
loan_data_strings

array([['5', '1', '36', '13', '1', '1'],
       ['0', '1', '36', '5', '1', '4'],
       ['9', '1', '36', '10', '1', '4'],
       ...,
       ['6', '1', '36', '5', '1', '1'],
       ['4', '1', '36', '17', '1', '3'],
       ['12', '1', '36', '4', '0', '3']], dtype='<U69')

In [921]:
loan_data_strings = loan_data_strings.astype(int)

In [922]:
loan_data_strings

array([[ 5,  1, 36, 13,  1,  1],
       [ 0,  1, 36,  5,  1,  4],
       [ 9,  1, 36, 10,  1,  4],
       ...,
       [ 6,  1, 36,  5,  1,  1],
       [ 4,  1, 36, 17,  1,  3],
       [12,  1, 36,  4,  0,  3]])

# Checkpoint

In [923]:
#Create a checkpoint 
checkpoint_strings = checkpoint("checkpoint_test10", header_strings, loan_data_strings)

In [924]:
checkpoint_strings["header"]

array(['issue_date', 'loan_status', 'term_months', 'sub_grade', 'verification_status',
       'state_address'], dtype='<U19')

In [925]:
checkpoint_strings["data"]

array([[ 5,  1, 36, 13,  1,  1],
       [ 0,  1, 36,  5,  1,  4],
       [ 9,  1, 36, 10,  1,  4],
       ...,
       [ 6,  1, 36,  5,  1,  1],
       [ 4,  1, 36, 17,  1,  3],
       [12,  1, 36,  4,  0,  3]])

# Clean and Preprocess Numeric columns

In [927]:
# Lets check first the missing values values 
loan_data_numeric

array([[48010226.  ,    35000.  ,    35000.  ,       13.33,     1184.86,     9452.96],
       [57693261.  ,    30000.  ,    30000.  , 68616520.  ,      938.57,     4679.7 ],
       [59432726.  ,    15000.  ,    15000.  , 68616520.  ,      494.86,     1969.83],
       ...,
       [50415990.  ,    10000.  ,    10000.  , 68616520.  , 68616520.  ,     2185.64],
       [46154151.  , 68616520.  ,    10000.  ,       16.55,      354.3 ,     3199.4 ],
       [66055249.  ,    10000.  ,    10000.  , 68616520.  ,      309.97,      301.9 ]])

# Substitute "Filler" values

In [930]:
np.isnan(loan_data_numeric).sum()

0

It looks like we don't have any missing numeric values, nut this isn't true because we had used a temporary filling when we had loaded our numeric data.

temporary_fill = np.nanmax(raw_data) + 1

we will need to substitue the temporary fill with the worst senario for our case


In [931]:
header_numeric

array(['id', 'loan_amnt', 'funded_amnt', 'int_rate', 'installment', 'total_pymnt'], dtype='<U19')

# We will check column by column 


# ID

In [939]:
np.unique(loan_data_numeric[:,0])

array([  373332.,   575239.,   707689., ..., 68614880., 68615915., 68616519.])

In [941]:
# temporary filler
temporary_fill

68616520.0

In [942]:
# temporary stats includes : min , mean, max values for each column
temporary_stats

array([[  373332.  ,         nan,     1000.  ,         nan,     1000.  ,         nan,        6.  ,
              31.42,         nan,         nan,         nan,         nan,         nan,        0.  ],
       [54015809.19,         nan,    15273.46,         nan,    15311.04,         nan,       16.62,
             440.92,         nan,         nan,         nan,         nan,         nan,     3143.85],
       [68616519.  ,         nan,    35000.  ,         nan,    35000.  ,         nan,       28.99,
            1372.97,         nan,         nan,         nan,         nan,         nan,    41913.62]])

In [943]:
# Lets first check and susbstitue the Filler with the worst possible value  

np.isin(loan_data_numeric[:,0], temporary_fill)

array([False, False, False, ..., False, False, False])

In [944]:
np.isin(loan_data_numeric[:,0], temporary_fill).sum()

0

There is no temporary fill in the ID column, no need to substitue anything

On the next step we will need to be carefull while choosing how to substitue the temporary fill, we need to choose the worst senario for each column values, min, mean or max value

In [945]:
header_numeric

array(['id', 'loan_amnt', 'funded_amnt', 'int_rate', 'installment', 'total_pymnt'], dtype='<U19')

In [957]:
# we have stored min, mean, and max values in temporary stats
# where the first row is the min for each column, 2nd row the mean and 3d row is the max for each column
temporary_stats[:, columns_numeric]

array([[  373332.  ,     1000.  ,     1000.  ,        6.  ,       31.42,        0.  ],
       [54015809.19,    15273.46,    15311.04,       16.62,      440.92,     3143.85],
       [68616519.  ,    35000.  ,    35000.  ,       28.99,     1372.97,    41913.62]])

In [949]:
# After carefull consideration we conclude to the following sbstitues : 
# 'funded_amnt': min filler value 
#'loan_amnt', 'int_rate', 'installment', 'total_pymnt' : max filler value 


In [960]:
# for example the min value for 'loan_amnt' is:
temporary_stats[:,columns_numeric][0,1]

1000.0

# Funded Amount

In [961]:
temporary_stats[0,columns_numeric[2]]

1000.0

In [963]:
# fill in funded amount column with the min value of funded amount column
loan_data_numeric[:,2] = np.where(loan_data_numeric[:,2] == temporary_fill, temporary_stats[0,columns_numeric[2]], loan_data_numeric[:,2] )
loan_data_numeric[:,2]

array([35000., 30000., 15000., ..., 10000., 10000., 10000.])

# 'loan_amnt', 'int_rate', 'installment', 'total_pymnt'

In [967]:
temporary_stats[2,columns_numeric[1]]

35000.0

In [968]:
# fill in the remaing column with the max values for their correspendant columns
#'loan_amnt'
loan_data_numeric[:,1] = np.where(loan_data_numeric[:,1] == temporary_fill, temporary_stats[2,columns_numeric[1]], loan_data_numeric[:,1] )
loan_data_numeric[:,1]

array([35000., 30000., 15000., ..., 10000., 35000., 10000.])

In [969]:
#'int_rate'
loan_data_numeric[:,3] = np.where(loan_data_numeric[:,3] == temporary_fill, temporary_stats[2,columns_numeric[3]], loan_data_numeric[:,3] )
loan_data_numeric[:,3]

array([13.33, 28.99, 28.99, ..., 28.99, 16.55, 28.99])

In [970]:
#'installment'
loan_data_numeric[:,4] = np.where(loan_data_numeric[:,4] == temporary_fill, temporary_stats[2,columns_numeric[4]], loan_data_numeric[:,4] )
loan_data_numeric[:,4]

array([1184.86,  938.57,  494.86, ..., 1372.97,  354.3 ,  309.97])

In [971]:
#'total_pymnt'
loan_data_numeric[:,5] = np.where(loan_data_numeric[:,5] == temporary_fill, temporary_stats[2,columns_numeric[5]], loan_data_numeric[:,5] )
loan_data_numeric[:,5]

array([9452.96, 4679.7 , 1969.83, ..., 2185.64, 3199.4 ,  301.9 ])

# Convert Dollars to Euros 

In [976]:
# The EUR-USD.csv file contains the average monthly exchange rate for 2015
EUR_USD = np.genfromtxt("EUR-USD.csv", 
                        delimiter = "," ,
                        autostrip =True,
                        dtype = str)
EUR_USD

array([['Open', 'High', 'Low', 'Close', 'Volume'],
       ['1.2098628282546997', '1.2098628282546997', '1.11055588722229', '1.1287955045700073', '0'],
       ['1.1287955045700073', '1.1484194993972778', '1.117680549621582', '1.1205360889434814',
        '0'],
       ['1.119795799255371', '1.1240400075912476', '1.0460032224655151', '1.0830246210098267',
        '0'],
       ['1.0741022825241089', '1.1247594356536865', '1.0521597862243652', '1.1114321947097778',
        '0'],
       ['1.1215037107467651', '1.145304799079895', '1.0821995735168457', '1.0960345268249512',
        '0'],
       ['1.095902442932129', '1.1428401470184326', '1.0888904333114624', '1.122296690940857', '0'],
       ['1.1134989261627197', '1.1219995021820068', '1.081270456314087', '1.0939244031906128',
        '0'],
       ['1.0969001054763794', '1.1705996990203857', '1.0850305557250977', '1.1340054273605347',
        '0'],
       ['1.1225990056991577', '1.1460003852844238', '1.1089695692062378', '1.1255937814712524

In [1020]:
# In CRM analysis we only use the Closing value of an exchange rate, thus we will
# only import the exchange rate we need
EUR_USD = np.genfromtxt("EUR-USD.csv", 
                        delimiter = ",",
                        autostrip =True,
                        skip_header = 1,
                        usecols = 3)
EUR_USD


array([1.13, 1.12, 1.08, 1.11, 1.1 , 1.12, 1.09, 1.13, 1.13, 1.1 , 1.06, 1.09])

In [1021]:
# the first column of our strings data is 'issue_date' which Correspondent to our exchage rate
loan_data_strings[:,0]

array([ 5,  0,  9, ...,  6,  4, 12])

In [1026]:
# Lets create a new variable which will contain the monthly Correspondent exchange rate
exchange_rate = loan_data_strings[:,0]
for i in range(1,13):
    exchange_rate = np.where(loan_data_strings[:,0] == i, 
              EUR_USD[i-1],
              loan_data_strings[:,0])
    
exchange_rate = np.where(exchange_rate == 0,
                         np.mean(EUR_USD),
                         exchange_rate)
exchange_rate 

array([5.  , 1.11, 9.  , ..., 6.  , 4.  , 1.09])

In [1027]:
 np.unique(exchange_rate)

array([ 1.  ,  1.09,  1.11,  2.  ,  3.  ,  4.  ,  5.  ,  6.  ,  7.  ,  8.  ,  9.  , 10.  , 11.  ])

In [1032]:
exchange_rate.shape

(10000,)

In [1033]:
loan_data_numeric.shape

(10000, 6)

In [ ]:
# We need to reshape exchange_rate, in order for us to insert it into loan_data_numeric


In [1035]:
exchange_rate = np.reshape(exchange_rate, (10000,1))

In [1036]:
loan_data_numeric = np.hstack((loan_data_numeric, exchange_rate))

In [1037]:
# We add the column name to the header
header_numeric = np.concatenate((header_numeric, np.array(['exchange_rate'])))
header_numeric

array(['id', 'loan_amnt', 'funded_amnt', 'int_rate', 'installment', 'total_pymnt', 'exchange_rate'],
      dtype='<U19')

### From the USD to EUR

In [1038]:
header_numeric

array(['id', 'loan_amnt', 'funded_amnt', 'int_rate', 'installment', 'total_pymnt', 'exchange_rate'],
      dtype='<U19')

In [1039]:
columns_dollar = np.array([1,2,4,5])

In [1040]:
loan_data_numeric[:,6]

array([5.  , 1.11, 9.  , ..., 6.  , 4.  , 1.09])

In [1041]:
# We will convert columns [1,2,4,5] into EUR by dividing each column by it respective exchange rate
for i in columns_dollar:
    loan_data_numeric = np.hstack((loan_data_numeric,
                                   np.reshape(loan_data_numeric[:,i] / loan_data_numeric[:,6],
                                    (10000,1))))

In [1042]:
loan_data_numeric.shape

(10000, 11)

In [1043]:
loan_data_numeric

array([[48010226.  ,    35000.  ,    35000.  , ...,     7000.  ,      236.97,     1890.59],
       [57693261.  ,    30000.  ,    30000.  , ...,    27132.46,      848.86,     4232.39],
       [59432726.  ,    15000.  ,    15000.  , ...,     1666.67,       54.98,      218.87],
       ...,
       [50415990.  ,    10000.  ,    10000.  , ...,     1666.67,      228.83,      364.27],
       [46154151.  ,    35000.  ,    10000.  , ...,     2500.  ,       88.58,      799.85],
       [66055249.  ,    10000.  ,    10000.  , ...,     9145.8 ,      283.49,      276.11]])

### Expanding the header

In [1044]:
header_additional = np.array([column_name + '_EUR' for column_name in header_numeric[columns_dollar]])

In [1045]:
header_additional

array(['loan_amnt_EUR', 'funded_amnt_EUR', 'installment_EUR', 'total_pymnt_EUR'], dtype='<U15')

In [1046]:
header_numeric = np.concatenate((header_numeric, header_additional))
header_numeric

array(['id', 'loan_amnt', 'funded_amnt', 'int_rate', 'installment', 'total_pymnt', 'exchange_rate',
       'loan_amnt_EUR', 'funded_amnt_EUR', 'installment_EUR', 'total_pymnt_EUR'], dtype='<U19')

In [1047]:
# we will need to sort the matrice column into a more meanfull order
columns_index_order = [0,1,7,2,8,3,4,9,5,10,6]

In [1049]:
header_numeric = header_numeric[columns_index_order]
loan_data_numeric

array([[48010226.  ,    35000.  ,    35000.  , ...,     7000.  ,      236.97,     1890.59],
       [57693261.  ,    30000.  ,    30000.  , ...,    27132.46,      848.86,     4232.39],
       [59432726.  ,    15000.  ,    15000.  , ...,     1666.67,       54.98,      218.87],
       ...,
       [50415990.  ,    10000.  ,    10000.  , ...,     1666.67,      228.83,      364.27],
       [46154151.  ,    35000.  ,    10000.  , ...,     2500.  ,       88.58,      799.85],
       [66055249.  ,    10000.  ,    10000.  , ...,     9145.8 ,      283.49,      276.11]])

In [1050]:
loan_data_numeric = loan_data_numeric[:,columns_index_order]

### Interest Rate

In [1052]:
header_numeric

array(['id', 'loan_amnt', 'installment_EUR', 'loan_amnt_EUR', 'total_pymnt', 'funded_amnt',
       'funded_amnt_EUR', 'total_pymnt_EUR', 'int_rate', 'exchange_rate', 'installment'],
      dtype='<U19')

In [1054]:
# when we check Interest Rate we notice its note between 0 and 1, it must be divided by 100
loan_data_numeric[:,5]

array([13.33, 28.99, 28.99, ..., 28.99, 16.55, 28.99])

In [1055]:
loan_data_numeric[:,5] = loan_data_numeric[:,5]/100

In [1056]:
loan_data_numeric[:,5]

array([0.13, 0.29, 0.29, ..., 0.29, 0.17, 0.29])

# Checkpoint : Numeric

In [1057]:
checkpoint_numeric = checkpoint("Checkpoint-Numeric", header_numeric, loan_data_numeric)

In [1058]:
checkpoint_numeric['header'], checkpoint_numeric['data']

(array(['id', 'loan_amnt', 'installment_EUR', 'loan_amnt_EUR', 'total_pymnt', 'funded_amnt',
        'funded_amnt_EUR', 'total_pymnt_EUR', 'int_rate', 'exchange_rate', 'installment'],
       dtype='<U19'),
 array([[48010226.  ,    35000.  ,     7000.  , ...,     9452.96,     1890.59,        5.  ],
        [57693261.  ,    30000.  ,    27132.46, ...,     4679.7 ,     4232.39,        1.11],
        [59432726.  ,    15000.  ,     1666.67, ...,     1969.83,      218.87,        9.  ],
        ...,
        [50415990.  ,    10000.  ,     1666.67, ...,     2185.64,      364.27,        6.  ],
        [46154151.  ,    35000.  ,     8750.  , ...,     3199.4 ,      799.85,        4.  ],
        [66055249.  ,    10000.  ,     9145.8 , ...,      301.9 ,      276.11,        1.09]]))

# Creating the "Complete" Dataset

In [1059]:
checkpoint_strings['data'].shape

(10000, 6)

In [1060]:
checkpoint_numeric['data'].shape

(10000, 11)

In [1065]:
# We stack the data string and numeric together 
loan_data = np.hstack((checkpoint_numeric['data'], checkpoint_strings['data']))

In [1062]:
loan_data

array([[48010226.  ,    35000.  ,     7000.  , ...,       13.  ,        1.  ,        1.  ],
       [57693261.  ,    30000.  ,    27132.46, ...,        5.  ,        1.  ,        4.  ],
       [59432726.  ,    15000.  ,     1666.67, ...,       10.  ,        1.  ,        4.  ],
       ...,
       [50415990.  ,    10000.  ,     1666.67, ...,        5.  ,        1.  ,        1.  ],
       [46154151.  ,    35000.  ,     8750.  , ...,       17.  ,        1.  ,        3.  ],
       [66055249.  ,    10000.  ,     9145.8 , ...,        4.  ,        0.  ,        3.  ]])

In [1063]:
np.isnan(loan_data).sum()

0

In [1066]:
# We stack the header string and numeric together
header_full = np.concatenate((checkpoint_numeric['header'], checkpoint_strings['header']))

# Sorting the New Dataset

In [1069]:
loan_data = loan_data[np.argsort(loan_data[:,0])]
loan_data

array([[  373332.  ,     9950.  ,      995.  , ...,       21.  ,        0.  ,        1.  ],
       [  575239.  ,    12000.  ,     1200.  , ...,       25.  ,        1.  ,        2.  ],
       [  707689.  ,    10000.  ,     5000.  , ...,       13.  ,        1.  ,        0.  ],
       ...,
       [68614880.  ,     5600.  ,     5121.65, ...,        8.  ,        1.  ,        1.  ],
       [68615915.  ,     4000.  ,     3658.32, ...,       10.  ,        1.  ,        2.  ],
       [68616519.  ,    21600.  ,    19754.93, ...,        3.  ,        0.  ,        2.  ]])

In [1070]:
np.argsort(loan_data[:,0])

array([   0,    1,    2, ..., 9997, 9998, 9999], dtype=int64)

# Storing the New Dataset

In [1071]:
loan_data = np.vstack((header_full, loan_data))

In [1073]:
np.savetxt("loan-data-preprocessed.csv", 
           loan_data, 
           fmt = '%s',
           delimiter = ',')